In [1]:
import numpy as np

## Introduction
When speedrunning Minecraft, players have to complete a couple tasks that rely on repeatedly doing the same thing until a random number generator gives the outcome they want. 

1. They try to get 7 blaze rods. Each time they kill a blaze, there is a 50% chance they receive a blaze rod.
2. They try to get 10-12 ender pearls. Each time they barter a piece of gold with a piglin, there is a 4.7% chance they receive a few ender pearls.

We would like to know how to calculate how lucky or unlucky a player is during a single run. If they kill 20 blazes and only get 6 blaze rods, they certainly were less lucky than average. Or if they gave a piglin 3 pieces of gold and got 2 sets of ender pearls, then they got very lucky. But we want to measure precisely how lucky. We want a formula to tell us what the chances of getting a certain amount of luck are. For example, "The probability of getting 8 blaze rods after killing 9 blazes is 1 in 1 in 200" or something (that number is fake).  

The Youtuber Dream killed blazes in 33 speedruns, killing 305 blazes and receiving 211 blaze rods. He traded with piglins in 22 speedruns, giving 262 pieces of gold and receiving 42 sets of ender pearls. Given that Dream got very lucky, we want an exact number to say how lucky.

## The Binomial Distribution
If every event is independent (the probability of a success does not change or depend on previous events), then the binomial distribution can calculate this exactly. In general, these events are independent. For example, after you kill a blaze, the next blaze still has a 50% chance of dropping a rod. The problem is that the speedrunner always stops killing blazes after getting the 7th blaze rod, and stops bartering gold after getting a set of ender pearls. This means that the last event is always a success. 

This has two possible implications:
1. The events are not truly independent, because the number of events depends on whether a certain number of successes occurred previously. This could mean that the binomial distribution will not give the correct probability.
2. The speedrunner will appear to have a higher rate of success than average, because ending with a success makes the proportion of successes higher than it should be.

## Coin-flipping Example
Suppose we are playing a game where the object of each round is to flip a coin until you get heads. The possible results of the game are:
* H (50% of the games)
* TH (25% of the games)
* TTH (12.5% of the games)

and so on. You can see that, on average, the proportion of heads will be higher in each game than the proportion of tails. In 50% of the games, 100% of the coin flips are heads! Let's simulate this game.

In [17]:
num_rounds = 20
total_flips = 0
proportions = []

# Play the game num_rounds times
for game_number in range(num_rounds):
    won = False
    num_flips = 0
    result_string = str(game_number + 1) + ". "
    
    # Keep flipping until we get a head
    while not won:
        result = np.random.random()
        num_flips += 1
        
        # If it is a head, end the round
        if result < 0.5:
            won = True
            total_flips += num_flips
            result_string += "H"
            prop_heads = int(100 / float(num_flips))
            proportions.append(1 / float(num_flips))
            
            # Print the result of the game
            print(result_string + " ({}%)".format(prop_heads))
        else:
            result_string += "T"

1. H (100%)
2. TTH (33%)
3. TTH (33%)
4. TH (50%)
5. H (100%)
6. H (100%)
7. TTTTTH (16%)
8. H (100%)
9. TTH (33%)
10. H (100%)
11. TH (50%)
12. H (100%)
13. H (100%)
14. TTTTH (20%)
15. H (100%)
16. TTH (33%)
17. H (100%)
18. H (100%)
19. H (100%)
20. TH (50%)


It sure looks like more rounds have high percentages of heads, right? Let's see the average percent:

In [18]:
print("Average percent of heads per round: " + str(100*sum(proportions) / len(proportions)))

Average percent of heads per round: 71.0


So this property of stopping on a success makes you look luckier than you actually are! But wait. What if we count the total number of heads and the total number of tails?

In [19]:
print("Total number of heads: {}".format(num_rounds))
print("Total number of tails: {}".format(total_flips - num_rounds))
print("Total percent of heads: {}".format(100 * num_rounds / float(total_flips)))

Total number of heads: 20
Total number of tails: 20
Total percent of heads: 50.0


That's much closer to 50%, isn't it? Why is average percent of heads per round so much higher than the total percent of heads? Shouldn't it be the same? This is somewhat of a paradox. It turns out that these averages are not weighted equally. When you have H followed by TTH, the percents are 100% and 33%, which average to 67%. But you really just had 2 heads and 2 tails, which is normal.  So while it is true that the percent of heads in each round will look higher on average, you are not actually getting more heads. You can just look at all of the rounds as a single list of coin flips. They are independent.

So when a speedrunner does a bunch of speedruns in a row, the fact that they stop each round after a success will not matter because they continue in the next speedrun.

## The Independence of the Events
Let's use Dream's blaze data as an example. Even though it is true that every blaze kill is independent, there is an argument to be made the number of blazes he killed was dependent on previous outcomes. The very last blaze he killed in his last speedrun is dependent on whether the blaze before it dropped a rod or not. If the blaze before it had dropped a rod, he would have had enough, and would not have continued killing more. And he would not have killed the second to last blaze if the third-to-last blaze had dropped a rod, and so on. There is a chain of dependence.  Some might argue that this means the binomial distribution cannot be applied.  However, the chain of dependence does not continue forever. Since Dream needed 7 blaze rods in his last speedrun, it was guaranteed that he would kill at least the first 7 blazes. This is not dependent on previous results. Only blazes 8, 9, and so on are dependent. So, we should not count blazes after the first 7 of the last speedrun.  So instead of seeing if Dream got lucky by getting 211 rods from 305 blazes, we can check if he got lucky by getting 210 rods from 303 blazes. This can be done with the binomial distribution.